In [1]:
import pickle
import pandas as pd
import numpy as np
import os
import time
from dotenv import load_dotenv

In [2]:
from openai import OpenAI
load_dotenv()
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [3]:
user_train_path = '../train_valid_test/user_train.pkl'
user_valid_path = '../train_valid_test/user_valid.pkl'
user_test_path = '../train_valid_test/user_test.pkl'

In [4]:
with open(user_train_path, 'rb') as f:
    user_train_dict = pickle.load(f)

In [5]:
with open(user_valid_path, 'rb') as f:
    user_valid_dict = pickle.load(f)

In [6]:
with open(user_test_path, 'rb') as f:
    user_test_dict = pickle.load(f)

In [9]:
with open('./rella_dataset/sem_sim_valid.pkl', 'rb') as f:
    user_sem_sim_valid_data = pickle.load(f)
print(len(user_sem_sim_valid_data), user_sem_sim_valid_data[1])

6040 [1022, 1270, 919, 938, 2018, 1028, 661, 3114, 1029, 2321, 1197, 594, 1, 588, 2687, 595, 2294, 783, 2355, 1907]


In [7]:
with open('./rella_dataset/sem_sim_test.pkl', 'rb') as f:
    user_sem_sim_test_data = pickle.load(f)
print(len(user_sem_sim_test_data), user_sem_sim_test_data[1])

6040 [1721, 1022, 919, 2018, 1028, 1246, 661, 531, 3114, 1029, 2321, 1197, 594, 1, 588, 2687, 595, 783, 1907, 1566]


In [8]:
user_id = 1
print(user_train_dict[user_id])
print(user_valid_dict[user_id])
print(user_test_dict[user_id])

[[1961, 2028, 3105, 938, 1962, 1035, 150, 2018, 1028, 1097, 914, 2797, 1287, 2762, 1246, 661, 2918, 531, 3114, 2791, 1029, 2321, 1197, 594, 2398, 1545, 527, 1, 588, 2687, 745, 595, 2294, 783, 2355, 1907]]
[1566]
[48]


In [9]:
len(user_train_dict[user_id][0])

36

In [10]:
user_profile_path = '../user_profile/cleaned_user_profile_mixtral.pkl'
with open(user_profile_path, 'rb') as f:
    user_profile_dict = pickle.load(f)
print(user_profile_dict[user_id])


            The user has a diverse taste in movies, with a preference for thought-provoking and emotionally engaging films. They enjoy dramas that explore complex themes such as mental illness, racial injustice, and the meaning of life. The user is drawn to movies that challenge their perspective and provoke deep thought, often favoring those that tackle serious and mature subjects.
            
            In addition to dramas, the user also has a soft spot for classic Disney films, as evidenced by their fondness for Cinderella. They appreciate the enchanting storytelling and timeless charm of these animated tales.
            
            The user's taste in movies also extends to comedies, as seen in their appreciation for the hilarious antics of Wallace & Gromit and the dark humor of Fargo. They are also drawn to adventure films, as demonstrated by their love for Star Wars: Episode IV - A New Hope and Back to the Future.
            
            However, the user is not a fan of 

In [11]:
movie_summary_df = pd.read_csv('../movie_summary_25_words.csv')
movie_summary_df.head()

,movie_id,name,cleaned_genre,year,summary
0,1,Toy Story,"Animation,Children's,Comedy",1995,Toy Story is a groundbreaking animated film th...
1,2,Jumanji,"Adventure,Children's,Fantasy",1995,"""Jumanji (1995) is a thrilling adventure film ..."
2,3,Grumpier Old Men,"Comedy,Romance",1995,"""Grumpier Old Men is a hilarious and heartwarm..."
3,4,Waiting to Exhale,"Comedy,Drama",1995,"""Waiting to Exhale"" (1995) is a heartfelt dram..."
4,5,Father of the Bride Part II,Comedy,1995,Father of the Bride Part II (1995) is a heartw...


In [12]:
print(movie_summary_df[movie_summary_df['movie_id'] == 265])

     movie_id                                               name  \
262       265  Like Water for Chocolate (Como agua para choco...   

     cleaned_genre  year                                            summary  
262  Drama,Romance  1992  "Like Water for Chocolate is a visually stunni...  


In [13]:
user_df = pd.read_csv('../users.csv')
user_df.head()

,user_id,user_profile
0,1,female k-12 student under 18 years of age
1,2,male self-employed above 56 years of age
2,3,male scientist between 25 to 34 years of age
3,4,male executive/managerial between 45 to 49 yea...
4,5,male writer between 25 to 34 years of age


In [14]:
ratings_df = pd.read_csv('../ratings.csv')
ratings_df.head()

,user_id,movie_id,rating
0,1,3186,4
1,1,1721,4
2,1,1022,5
3,1,1270,5
4,1,2340,3


In [19]:
crt_sem_valid_zeroshot_dataset = dict()
cnt = 0
for user, user_profile in user_profile_dict.items():
    cnt += 1
    final_user_profile = "The user is a "+user_df[user_df['user_id'] == user]['user_profile'].values[0]+"\n"
    # print(final_user_profile)
    prompt = final_user_profile
    prompt += """
   User watched the following movies in the given order in the past and rated them -
    """
    # print(prompt)
    user_rating_dict = ratings_df[ratings_df['user_id'] == user]
    # print(user_rating_dict)
    liked_movies = []
    disliked_movies = []

    prompt += "["
    for movie in user_sem_sim_test_data[user]:
        # print(movie)
        user_movie_rating = user_rating_dict[user_rating_dict['movie_id'] == movie]['rating'].values[0]
        # print(movie, user_movie_rating)
        movie_details = movie_summary_df[movie_summary_df['movie_id'] == movie]
        prompt += movie_details['name'].values[0] + " (" + str(movie_details['year'].values[0]) + ") (" + str(user_movie_rating) + " stars), "
    prompt += "]"

    ### To predict
    # print(user_valid_dict[user][0])
    movie_details = movie_summary_df[movie_summary_df['movie_id'] == user_test_dict[user][0]]
    prompt += f"""
    Based on the movies he has watched, deduce if he will like the movie {movie_details['name'].values[0]} ({str(movie_details['year'].values[0])}). Note that more stars the user rated the movie, the user like the movie more.
    You should ONLY tell me Yes or No in the following format -
    Prediction: Yes or No
    """
    # print(f"Prompt length: {len(prompt.split())}", prompt)
    crt_sem_valid_zeroshot_dataset[user] = prompt
    # break

In [20]:
print(len(crt_sem_valid_zeroshot_dataset))

6040


In [21]:
crt_sem_valid_zeroshot_dataset[124]

"The user is a male executive/managerial above 56 years of age\n\n   User watched the following movies in the given order in the past and rated them -\n    [Cape Fear  (1991) (4 stars), Lost World: Jurassic Park, The  (1997) (3 stars), Fatal Attraction  (1987) (4 stars), My Cousin Vinny  (1992) (4 stars), Backdraft  (1991) (4 stars), Washington Square  (1997) (5 stars), Schindler's List  (1993) (4 stars), American Beauty  (1999) (3 stars), Sling Blade  (1996) (5 stars), Thelma & Louise  (1991) (5 stars), Fugitive, The  (1993) (3 stars), Liberty Heights  (1999) (5 stars), Amistad  (1997) (4 stars), Some Folks Call It a Sling Blade  (1993) (5 stars), Limey, The  (1999) (4 stars), GoodFellas  (1990) (4 stars), Crying Game, The  (1992) (4 stars), Babe  (1995) (5 stars), Grifters, The  (1990) (5 stars), End of the Affair, The  (1999) (4 stars), ]\n    Based on the movies he has watched, deduce if he will like the movie Boiler Room  (2000). Note that more stars the user rated the movie, the 

In [22]:
crt_sem_valid_zeroshot_dataset_path = './rella_dataset/rella_test_dataset.pkl'
with open(crt_sem_valid_zeroshot_dataset_path, 'wb') as f:
    pickle.dump(crt_sem_valid_zeroshot_dataset, f)
    f.close()

In [37]:
# with open(crt_valid_zeroshot_dataset_path, 'rb') as f:
#     crt_valid_zeroshot_dataset_temp = pickle.load(f)
# print(len(crt_valid_zeroshot_dataset_temp), crt_valid_zeroshot_dataset_temp[1])

In [23]:
content = prompt
print(crt_sem_valid_zeroshot_dataset[1])
print('*'*100)
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "user",
      "content": crt_sem_valid_zeroshot_dataset[1]
    }
  ],
  temperature=0.1,
  max_tokens=64,
  top_p=0.9
)

response = response.choices[0].message.content
print(response)
len(response.split(' '))

The user is a female k-12 student under 18 years of age

   User watched the following movies in the given order in the past and rated them -
    [Titanic  (1997) (4 stars), Cinderella  (1950) (5 stars), Wizard of Oz, The  (1939) (4 stars), Bambi  (1942) (4 stars), Mary Poppins  (1964) (5 stars), Dead Poets Society  (1989) (4 stars), James and the Giant Peach  (1996) (3 stars), Secret Garden, The  (1993) (4 stars), Toy Story 2  (1999) (4 stars), Dumbo  (1941) (5 stars), Pleasantville  (1998) (3 stars), Princess Bride, The  (1987) (3 stars), Snow White and the Seven Dwarfs  (1937) (4 stars), Toy Story  (1995) (5 stars), Aladdin  (1992) (4 stars), Tarzan  (1999) (3 stars), Beauty and the Beast  (1991) (5 stars), Hunchback of Notre Dame, The  (1996) (4 stars), Mulan  (1998) (4 stars), Hercules  (1997) (4 stars), ]
    Based on the movies he has watched, deduce if he will like the movie Pocahontas  (1995). Note that more stars the user rated the movie, the user like the movie more.
    You

2